In [1]:
"""
Model Diagnostics - Feature Importance Analysis
================================================

Purpose: Validate feature engineering quality by measuring predictive power
         of features for both P1A (poor performance) and P3A (excellent).

Key Question: Do P1A features have ANY predictive power, or is the
              Atlantic market fundamentally unpredictable?

Methods:
1. Permutation Importance (model-agnostic)
2. XGBoost Built-in Feature Importance (gain, cover, weight)
3. SHAP Values (Shapley Additive Explanations)

Expected Outcomes:
- If P1A features show near-zero importance → Feature engineering needs work
- If P1A features show positive importance → Market is unpredictable

Author: Data Science Pipeline
Date: 2025-10-17
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import shap
import warnings
import json
import os

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# ==============================================================================
# CONFIGURATION
# ==============================================================================

DATA_DIR = 'data/processed/'
MODELS_DIR = 'data/models/xgboost_core/'
OUTPUT_DIR = 'data/diagnostics/'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}plots/', exist_ok=True)

HORIZON = 1  # Focus on h=1 for diagnostics
N_REPEATS = 10  # Permutation importance repetitions
RANDOM_STATE = 73
TOP_N_FEATURES = 12  # Show all 12 CORE features

print('='*80)
print('MODEL DIAGNOSTICS: FEATURE IMPORTANCE ANALYSIS')
print('='*80)
print('Objective: Determine if poor P1A performance is due to:')
print('  A) Poor feature engineering (features lack predictive power)')
print('  B) Market unpredictability (features are good but market is chaotic)')
print('='*80)
print(f'\nConfiguration:')
print(f'  Data directory: {DATA_DIR}')
print(f'  Models directory: {MODELS_DIR}')
print(f'  Output directory: {OUTPUT_DIR}')
print(f'  Analysis horizon: h={HORIZON}')
print(f'  Permutation repeats: {N_REPEATS}')

# ==============================================================================
# LOAD DATA AND MODELS
# ==============================================================================

print('\n' + '='*80)
print('LOADING DATA AND MODELS')
print('='*80)

# Load CORE features (12 features per route)
p1a_train = pd.read_csv(f'{DATA_DIR}p1a_core_train.csv')
p1a_val = pd.read_csv(f'{DATA_DIR}p1a_core_val.csv')
p1a_test = pd.read_csv(f'{DATA_DIR}p1a_core_test.csv')

p3a_train = pd.read_csv(f'{DATA_DIR}p3a_core_train.csv')
p3a_val = pd.read_csv(f'{DATA_DIR}p3a_core_val.csv')
p3a_test = pd.read_csv(f'{DATA_DIR}p3a_core_test.csv')

# Load targets
targets_train = pd.read_csv(f'{DATA_DIR}targets_train.csv')
targets_val = pd.read_csv(f'{DATA_DIR}targets_val.csv')
targets_test = pd.read_csv(f'{DATA_DIR}targets_test.csv')

print(f'[LOADED] P1A: Train={p1a_train.shape}, Val={p1a_val.shape}, Test={p1a_test.shape}')
print(f'[LOADED] P3A: Train={p3a_train.shape}, Val={p3a_val.shape}, Test={p3a_test.shape}')
print(f'[LOADED] Targets: Train={targets_train.shape}, Val={targets_val.shape}, Test={targets_test.shape}')

# Feature columns
p1a_features = [c for c in p1a_train.columns if c != 'Date']
p3a_features = [c for c in p3a_train.columns if c != 'Date']

print(f'\nFeature counts: P1A={len(p1a_features)}, P3A={len(p3a_features)}')

# Load trained XGBoost models
print('\n[LOADING] XGBoost models...')
p1a_model = xgb.XGBRegressor()
p1a_model.load_model(f'{MODELS_DIR}P1A_h{HORIZON}_core_model.json')
print(f'  OK P1A model loaded')

p3a_model = xgb.XGBRegressor()
p3a_model.load_model(f'{MODELS_DIR}P3A_h{HORIZON}_core_model.json')
print(f'  OK P3A model loaded')

# ==============================================================================
# PREPARE CLEAN DATASETS
# ==============================================================================

print('\n' + '='*80)
print('PREPARING CLEAN DATASETS')
print('='*80)

def prepare_clean_data(X_train, X_val, y_train, y_val, feature_names, route_name):
    """Prepare clean dataset by removing NaN values."""
    train_mask = ~y_train.isna()
    val_mask = ~y_val.isna()

    X_train_clean = X_train[feature_names][train_mask].reset_index(drop=True)
    y_train_clean = y_train[train_mask].reset_index(drop=True)

    X_val_clean = X_val[feature_names][val_mask].reset_index(drop=True)
    y_val_clean = y_val[val_mask].reset_index(drop=True)

    # Combine for robust importance estimation
    X_combined = pd.concat([X_train_clean, X_val_clean], axis=0, ignore_index=True)
    y_combined = pd.concat([y_train_clean, y_val_clean], axis=0, ignore_index=True)

    print(f'{route_name}:')
    print(f'  Train samples: {len(y_train_clean)}')
    print(f'  Val samples:   {len(y_val_clean)}')
    print(f'  Combined:      {len(y_combined)} (used for importance analysis)')

    return X_combined, y_combined, X_val_clean, y_val_clean

# P1A data
X_p1a, y_p1a, X_p1a_val, y_p1a_val = prepare_clean_data(
    p1a_train, p1a_val,
    targets_train[f'P1A_82_h{HORIZON}'],
    targets_val[f'P1A_82_h{HORIZON}'],
    p1a_features, 'P1A_82'
)

print()

# P3A data
X_p3a, y_p3a, X_p3a_val, y_p3a_val = prepare_clean_data(
    p3a_train, p3a_val,
    targets_train[f'P3A_82_h{HORIZON}'],
    targets_val[f'P3A_82_h{HORIZON}'],
    p3a_features, 'P3A_82'
)

# ==============================================================================
# BASELINE MODEL PERFORMANCE
# ==============================================================================

print('\n' + '='*80)
print('BASELINE MODEL PERFORMANCE (Validation Set)')
print('='*80)

def evaluate_model(model, X_val, y_val, route_name):
    """Evaluate model performance on validation set."""
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    print(f'\n{route_name}:')
    print(f'  RMSE: ${rmse:,.2f}')
    print(f'  MAE:  ${mae:,.2f}')
    print(f'  R2:   {r2:.4f}')
    if r2 < 0:
        print(f'  WARNING: Negative R2 indicates predictions worse than mean baseline')

    return {'rmse': rmse, 'mae': mae, 'r2': r2}

p1a_perf = evaluate_model(p1a_model, X_p1a_val, y_p1a_val, 'P1A_82')
p3a_perf = evaluate_model(p3a_model, X_p3a_val, y_p3a_val, 'P3A_82')

# ==============================================================================
# METHOD 1: PERMUTATION IMPORTANCE
# ==============================================================================

print('\n' + '='*80)
print('METHOD 1: PERMUTATION IMPORTANCE')
print('='*80)
print(f'Computing permutation importance ({N_REPEATS} repeats per feature)...')
print('This may take a few minutes...')

# P1A Permutation Importance
print('\n[P1A_82] Computing permutation importance...')
p1a_perm = permutation_importance(
    p1a_model, X_p1a_val, y_p1a_val,
    n_repeats=N_REPEATS,
    random_state=RANDOM_STATE,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

p1a_perm_df = pd.DataFrame({
    'feature': p1a_features,
    'importance_mean': -p1a_perm.importances_mean,  # Negate because neg_rmse
    'importance_std': p1a_perm.importances_std
}).sort_values('importance_mean', ascending=False)

print(f'  OK Complete')
print(f'  Top feature: {p1a_perm_df.iloc[0]["feature"]} (importance: {p1a_perm_df.iloc[0]["importance_mean"]:.2f})')
print(f'  Positive importance features: {(p1a_perm_df["importance_mean"] > 0).sum()} / {len(p1a_features)}')

# P3A Permutation Importance
print('\n[P3A_82] Computing permutation importance...')
p3a_perm = permutation_importance(
    p3a_model, X_p3a_val, y_p3a_val,
    n_repeats=N_REPEATS,
    random_state=RANDOM_STATE,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

p3a_perm_df = pd.DataFrame({
    'feature': p3a_features,
    'importance_mean': -p3a_perm.importances_mean,
    'importance_std': p3a_perm.importances_std
}).sort_values('importance_mean', ascending=False)

print(f'  OK Complete')
print(f'  Top feature: {p3a_perm_df.iloc[0]["feature"]} (importance: {p3a_perm_df.iloc[0]["importance_mean"]:.2f})')
print(f'  Positive importance features: {(p3a_perm_df["importance_mean"] > 0).sum()} / {len(p3a_features)}')

# Display results
print('\n' + '='*80)
print('PERMUTATION IMPORTANCE RESULTS')
print('='*80)

print('\n[P1A_82] Top Features:')
print('-'*80)
for i, row in p1a_perm_df.head(10).iterrows():
    print(f"  {row['feature'][:50]:50s}: {row['importance_mean']:>8.2f} +/- {row['importance_std']:.2f}")

print('\n[P3A_82] Top Features:')
print('-'*80)
for i, row in p3a_perm_df.head(10).iterrows():
    print(f"  {row['feature'][:50]:50s}: {row['importance_mean']:>8.2f} +/- {row['importance_std']:.2f}")

# Critical analysis
print('\n' + '='*80)
print('CRITICAL ANALYSIS')
print('='*80)

p1a_max_importance = p1a_perm_df['importance_mean'].max()
p3a_max_importance = p3a_perm_df['importance_mean'].max()
p1a_positive_count = (p1a_perm_df['importance_mean'] > 0).sum()
p3a_positive_count = (p3a_perm_df['importance_mean'] > 0).sum()

print(f'\nP1A Feature Quality:')
print(f'  Maximum importance: {p1a_max_importance:.2f}')
print(f'  Features with positive importance: {p1a_positive_count} / {len(p1a_features)}')
print(f'  Mean importance (all features): {p1a_perm_df["importance_mean"].mean():.2f}')

print(f'\nP3A Feature Quality:')
print(f'  Maximum importance: {p3a_max_importance:.2f}')
print(f'  Features with positive importance: {p3a_positive_count} / {len(p3a_features)}')
print(f'  Mean importance (all features): {p3a_perm_df["importance_mean"].mean():.2f}')

print('\n' + '-'*80)
print('DIAGNOSIS:')
if p1a_max_importance > 50:
    print('OK P1A features DO have predictive power')
    print('   -> Poor performance is due to MARKET UNPREDICTABILITY, not feature quality')
    diagnosis = 'Features are good, market is unpredictable'
else:
    print('WARNING P1A features show weak predictive power')
    print('   -> Consider revising feature engineering or adding new features')
    diagnosis = 'Features may need improvement'

importance_ratio = p3a_max_importance / p1a_max_importance if p1a_max_importance > 0 else float('inf')
print(f'\nP3A vs P1A importance ratio: {importance_ratio:.2f}x')

# Plot permutation importance
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# P1A
top_p1a = p1a_perm_df.head(TOP_N_FEATURES)
axes[0].barh(range(len(top_p1a)), top_p1a['importance_mean'],
             xerr=top_p1a['importance_std'], alpha=0.7, color='darkred')
axes[0].set_yticks(range(len(top_p1a)))
axes[0].set_yticklabels(top_p1a['feature'], fontsize=9)
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance (RMSE increase when permuted)', fontsize=11)
axes[0].set_title(f'P1A_82 - Permutation Importance', fontsize=12, fontweight='bold')
axes[0].axvline(0, color='black', linestyle='--', linewidth=1)
axes[0].grid(True, alpha=0.3, axis='x')

# P3A
top_p3a = p3a_perm_df.head(TOP_N_FEATURES)
axes[1].barh(range(len(top_p3a)), top_p3a['importance_mean'],
             xerr=top_p3a['importance_std'], alpha=0.7, color='darkblue')
axes[1].set_yticks(range(len(top_p3a)))
axes[1].set_yticklabels(top_p3a['feature'], fontsize=9)
axes[1].invert_yaxis()
axes[1].set_xlabel('Importance (RMSE increase when permuted)', fontsize=11)
axes[1].set_title(f'P3A_82 - Permutation Importance', fontsize=12, fontweight='bold')
axes[1].axvline(0, color='black', linestyle='--', linewidth=1)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}plots/permutation_importance.png', dpi=150, bbox_inches='tight')
plt.close()
print(f'\n[SAVED] plots/permutation_importance.png')

# ==============================================================================
# METHOD 2: XGBOOST FEATURE IMPORTANCE
# ==============================================================================

print('\n' + '='*80)
print('METHOD 2: XGBOOST BUILT-IN FEATURE IMPORTANCE')
print('='*80)

def get_xgb_importance(model, feature_names):
    """Extract XGBoost importance scores."""
    gain = model.get_booster().get_score(importance_type='gain')
    cover = model.get_booster().get_score(importance_type='cover')
    weight = model.get_booster().get_score(importance_type='weight')

    importance_df = pd.DataFrame({
        'feature': feature_names,
        'gain': [gain.get(f'f{i}', 0) for i in range(len(feature_names))],
        'cover': [cover.get(f'f{i}', 0) for i in range(len(feature_names))],
        'weight': [weight.get(f'f{i}', 0) for i in range(len(feature_names))]
    })

    return importance_df

# Extract importance
print('\n[P1A_82] Extracting XGBoost importance scores...')
p1a_xgb_imp = get_xgb_importance(p1a_model, p1a_features)
p1a_xgb_imp_gain = p1a_xgb_imp.sort_values('gain', ascending=False)
print(f'  OK Complete')
print(f'  Top feature (by gain): {p1a_xgb_imp_gain.iloc[0]["feature"]} ({p1a_xgb_imp_gain.iloc[0]["gain"]:.2f})')

print('\n[P3A_82] Extracting XGBoost importance scores...')
p3a_xgb_imp = get_xgb_importance(p3a_model, p3a_features)
p3a_xgb_imp_gain = p3a_xgb_imp.sort_values('gain', ascending=False)
print(f'  OK Complete')
print(f'  Top feature (by gain): {p3a_xgb_imp_gain.iloc[0]["feature"]} ({p3a_xgb_imp_gain.iloc[0]["gain"]:.2f})')

# Display
print('\n[P1A_82] Top Features (by Gain):')
print('-'*80)
for i, row in p1a_xgb_imp_gain.head(10).iterrows():
    print(f"  {row['feature'][:50]:50s}: Gain={row['gain']:>8.2f}")

print('\n[P3A_82] Top Features (by Gain):')
print('-'*80)
for i, row in p3a_xgb_imp_gain.head(10).iterrows():
    print(f"  {row['feature'][:50]:50s}: Gain={row['gain']:>8.2f}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# P1A
top_p1a_xgb = p1a_xgb_imp_gain.head(TOP_N_FEATURES)
axes[0].barh(range(len(top_p1a_xgb)), top_p1a_xgb['gain'], alpha=0.7, color='darkred')
axes[0].set_yticks(range(len(top_p1a_xgb)))
axes[0].set_yticklabels(top_p1a_xgb['feature'], fontsize=9)
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance (Gain)', fontsize=11)
axes[0].set_title(f'P1A_82 - XGBoost Gain', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# P3A
top_p3a_xgb = p3a_xgb_imp_gain.head(TOP_N_FEATURES)
axes[1].barh(range(len(top_p3a_xgb)), top_p3a_xgb['gain'], alpha=0.7, color='darkblue')
axes[1].set_yticks(range(len(top_p3a_xgb)))
axes[1].set_yticklabels(top_p3a_xgb['feature'], fontsize=9)
axes[1].invert_yaxis()
axes[1].set_xlabel('Importance (Gain)', fontsize=11)
axes[1].set_title(f'P3A_82 - XGBoost Gain', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}plots/xgboost_importance_gain.png', dpi=150, bbox_inches='tight')
plt.close()
print(f'\n[SAVED] plots/xgboost_importance_gain.png')

# ==============================================================================
# METHOD 3: SHAP VALUES
# ==============================================================================

print('\n' + '='*80)
print('METHOD 3: SHAP VALUES (Shapley Additive Explanations)')
print('='*80)
print('Computing SHAP values (this may take several minutes)...')

# P1A SHAP
print('\n[P1A_82] Computing SHAP values...')
p1a_explainer = shap.TreeExplainer(p1a_model)
p1a_shap_values = p1a_explainer.shap_values(X_p1a_val)
p1a_shap_importance = pd.DataFrame({
    'feature': p1a_features,
    'shap_importance': np.abs(p1a_shap_values).mean(axis=0)
}).sort_values('shap_importance', ascending=False)
print(f'  OK Complete')
print(f'  Top feature: {p1a_shap_importance.iloc[0]["feature"]} (SHAP: {p1a_shap_importance.iloc[0]["shap_importance"]:.2f})')

# P3A SHAP
print('\n[P3A_82] Computing SHAP values...')
p3a_explainer = shap.TreeExplainer(p3a_model)
p3a_shap_values = p3a_explainer.shap_values(X_p3a_val)
p3a_shap_importance = pd.DataFrame({
    'feature': p3a_features,
    'shap_importance': np.abs(p3a_shap_values).mean(axis=0)
}).sort_values('shap_importance', ascending=False)
print(f'  OK Complete')
print(f'  Top feature: {p3a_shap_importance.iloc[0]["feature"]} (SHAP: {p3a_shap_importance.iloc[0]["shap_importance"]:.2f})')

# SHAP summary plots
plt.figure(figsize=(12, 8))
shap.summary_plot(p1a_shap_values, X_p1a_val, feature_names=p1a_features,
                  max_display=TOP_N_FEATURES, show=False)
plt.title(f'P1A_82 - SHAP Feature Importance', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}plots/shap_summary_p1a.png', dpi=150, bbox_inches='tight')
plt.close()
print(f'[SAVED] plots/shap_summary_p1a.png')

plt.figure(figsize=(12, 8))
shap.summary_plot(p3a_shap_values, X_p3a_val, feature_names=p3a_features,
                  max_display=TOP_N_FEATURES, show=False)
plt.title(f'P3A_82 - SHAP Feature Importance', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}plots/shap_summary_p3a.png', dpi=150, bbox_inches='tight')
plt.close()
print(f'[SAVED] plots/shap_summary_p3a.png')

# ==============================================================================
# SAVE RESULTS
# ==============================================================================

print('\n' + '='*80)
print('SAVING RESULTS')
print('='*80)

# Save importance tables
p1a_perm_df.to_csv(f'{OUTPUT_DIR}p1a_permutation_importance.csv', index=False)
p3a_perm_df.to_csv(f'{OUTPUT_DIR}p3a_permutation_importance.csv', index=False)
print(f'[SAVED] Permutation importance CSVs')

p1a_xgb_imp_gain.to_csv(f'{OUTPUT_DIR}p1a_xgboost_importance.csv', index=False)
p3a_xgb_imp_gain.to_csv(f'{OUTPUT_DIR}p3a_xgboost_importance.csv', index=False)
print(f'[SAVED] XGBoost importance CSVs')

p1a_shap_importance.to_csv(f'{OUTPUT_DIR}p1a_shap_importance.csv', index=False)
p3a_shap_importance.to_csv(f'{OUTPUT_DIR}p3a_shap_importance.csv', index=False)
print(f'[SAVED] SHAP importance CSVs')

# Save summary
summary = {
    'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'horizon': HORIZON,
    'p1a_performance': {
        'rmse': float(p1a_perf['rmse']),
        'mae': float(p1a_perf['mae']),
        'r2': float(p1a_perf['r2'])
    },
    'p3a_performance': {
        'rmse': float(p3a_perf['rmse']),
        'mae': float(p3a_perf['mae']),
        'r2': float(p3a_perf['r2'])
    },
    'p1a_max_importance': {
        'permutation': float(p1a_max_importance),
        'xgboost_gain': float(p1a_xgb_imp_gain['gain'].max()),
        'shap': float(p1a_shap_importance['shap_importance'].max())
    },
    'p3a_max_importance': {
        'permutation': float(p3a_max_importance),
        'xgboost_gain': float(p3a_xgb_imp_gain['gain'].max()),
        'shap': float(p3a_shap_importance['shap_importance'].max())
    },
    'diagnosis': diagnosis,
    'importance_ratio_p3a_vs_p1a': float(importance_ratio)
}

with open(f'{OUTPUT_DIR}diagnostics_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print(f'[SAVED] diagnostics_summary.json')

print('\n' + '='*80)
print('DIAGNOSTICS COMPLETE!')
print('='*80)
print(f'\nAll results saved to: {OUTPUT_DIR}')
print('\nKey files:')
print('  - diagnostics_summary.json: Summary of findings')
print('  - p1a/p3a_*_importance.csv: Detailed importance scores')
print('  - plots/: Visualizations for all three methods')
print('\n' + '='*80)
print('\nFINAL ANSWER TO YOUR QUESTION:')
print(f'  "{diagnosis}"')
if p1a_max_importance > 50:
    print('\n  DO NOT change feature engineering.')
    print('  The features are working. The Atlantic market is simply unpredictable.')
else:
    print('\n  CONSIDER revising feature engineering.')
    print('  Features show weak predictive power.')
print('='*80)


MODEL DIAGNOSTICS: FEATURE IMPORTANCE ANALYSIS
Objective: Determine if poor P1A performance is due to:
  A) Poor feature engineering (features lack predictive power)
  B) Market unpredictability (features are good but market is chaotic)

Configuration:
  Data directory: data/processed/
  Models directory: data/models/xgboost_core/
  Output directory: data/diagnostics/
  Analysis horizon: h=1
  Permutation repeats: 10

LOADING DATA AND MODELS
[LOADED] P1A: Train=(705, 13), Val=(125, 13), Test=(326, 13)
[LOADED] P3A: Train=(705, 13), Val=(125, 13), Test=(326, 13)
[LOADED] Targets: Train=(705, 11), Val=(125, 11), Test=(326, 11)

Feature counts: P1A=12, P3A=12

[LOADING] XGBoost models...
  OK P1A model loaded
  OK P3A model loaded

PREPARING CLEAN DATASETS
P1A_82:
  Train samples: 705
  Val samples:   125
  Combined:      830 (used for importance analysis)

P3A_82:
  Train samples: 705
  Val samples:   125
  Combined:      830 (used for importance analysis)

BASELINE MODEL PERFORMANCE (Va